In [ ]:
%load_ext autoreload
%autoreload 2

## Lession 3: Neural Net Foundations

> mathematical foundations of deep learning: Stochastic gradient descent (SGD), and the flexibility of linear functions layered with non-linear activation functions. 

In [ ]:
#| default_exp neuralnet

## Initial Checks

In [ ]:
!conda list | grep "pytorch"

ffmpeg                    4.3                  hf484d3e_0    pytorch
pytorch                   2.0.1           py3.11_cuda11.8_cudnn8.7.0_0    pytorch
pytorch-cuda              11.8                 h7e8668a_5    pytorch
pytorch-ignite            0.4.12                   pypi_0    pypi
pytorch-lightning         2.0.6                    pypi_0    pypi
pytorch-mutex             1.0                        cuda    pytorch
torchaudio                2.0.2               py311_cu118    pytorch
torchtriton               2.0.0                     py311    pytorch
torchvision               0.15.2              py311_cu118    pytorch


In [ ]:
!pip list | grep "fastai" 
!pip list | grep "fastbook"
!pip list | grep "ipywidgets"
!pip list | grep "fastAIcourse"

fastai                        2.7.12
fastbook                      0.0.29
ipywidgets                    7.7.5
fastAIcourse                  0.0.1     /home/ben/BENEDICT_Only/Benedict_Projects/Benedict_ML/fastAIcourse


In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

## Gather Data

In [ ]:
from fastbook import search_images_ddg
from fastdownload import download_url
from fastai.vision.all import *
import os

In [ ]:
from fastAIcourse.intro import *

In [ ]:
!pip list | grep 'timm'

In [ ]:
import timm 

In [ ]:
timm.list_models('convnext*')